In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
from pandas.io.json import json_normalize
import glob, os, json, pickle
import re

import cv2, face_recognition

# Read in Data

In [ ]:
og_ad = pd.read_csv("./data/og_ad.csv", encoding = "ISO-8859-1")
og_ad.head()

In [ ]:
with open("./output/candidate_face_encodings.json", "r") as f:
    candidate_face_encodings = json.load(f)

known_names = []
known_faces = []
for k, face in candidate_face_encodings.items():
    if face['face_encoding'] is not None:
        known_names.append(k)
        known_faces.append(face['face_encoding'])
        

# Run script on all

In [ ]:
media_fldr = "./data/og_ad_media/*"
ims = "./data/og_ad_media/*.jpg"
vids = "./data/og_ad_media/*.mp4"
media_fps = glob.glob(media_fldr)
print(len(media_fps), len(glob.glob(ims)), len(glob.glob(vids)))


In [ ]:
# List all our media files by ID
media_dict = {}
for m in media_fps:
    media_name = os.path.basename(m)
    x = media_name.split("_")
    ad_id, media_type = x[0], x[1]
    
    images = []
    videos = []
    
    if "image" in media_type:
        images.append(m)
    else:
        videos.append(m)
    
    if ad_id in media_dict:
        media_dict[ad_id]['images'].extend(images)
        media_dict[ad_id]['videos'].extend(videos)
    else:
        media_dict[ad_id] = {
            "images": images,
            "videos": videos
        }
    

In [ ]:
with open('./output/og_medias.json', 'w') as fp:
    json.dump(media_dict, fp, sort_keys=True, indent=4)

In [ ]:
ad = media_dict["1008372119347745"]
print(ad)
ad_im = ad['images'][1]
# ad_vid = ad['videos'][0]


In [ ]:
def grab_media_name(fp):
    names = re.findall("[A-Za-z0-9_]+\.[a-z0-9]{3}", fp)[0]
    return names
#     return f"{names[0]}_{names[1]}.{names[2]}"
grab_media_name("./data/og_ad_media\\1000052250196513_image1.jpg")
grab_media_name("./data/og_ad_media\\1000869713437051_video0.mp4")


In [ ]:
import recognize_faces as recog
rec_img = recog.FaceRecognizer(known_names, known_faces)
vid_annot = recog.VideoAnnotator(known_names, known_faces, 5)

In [ ]:
# Find all mentions in images and videos
media_mentions = {}
for k, medias in media_dict.items():
    ims, vids = medias['images'], medias['videos']    
    im_mentions, vid_mentions = [], []
    
    for im in ims:
        rgb_img = rec_img.read_image_rgb(im)
        mentions = []
        if rgb_img is not None:
            found_faces = rec_img.name_all_faces(rgb_img, draw_matches=True)
            mentions = found_faces['face_names']
            plt.imshow(found_faces['boxed_img'])
            plt.show()
        im_mentions.append((im, mentions))
    for vid in vids:
        vid_annot.process_vid(vid)
        vid_mentions.append((vid, ['Fred', "Fred2"]))
    
    media_mentions[k] = {
        'images': im_mentions,
        'videos': vid_mentions,
    }

In [ ]:
media_mentions

### Convert to DataFrame

In [ ]:
mentions_dict = {
    'ad_id': [],
    'media_type': [],
    'loc': [],
    'mentions': [],
}

In [ ]:
for k, medias in media_mentions.items():
    ims_mentions, vids_mentions = medias['images'], medias['videos']    

    for (im_loc, im_mention) in ims_mentions:
        for m in im_mention:
            mentions_dict['ad_id'].append(k)
            mentions_dict['media_type'].append("image")
            mentions_dict['loc'].append(grab_media_name(im_loc))
            mentions_dict['mentions'].append(m)
    for (vid_loc, vid_mention) in vid_mentions:
        for m in vid_mention:
            mentions_dict['ad_id'].append(k)
            mentions_dict['media_type'].append("video")
            mentions_dict['loc'].append(grab_media_name(vid_loc))
            mentions_dict['mentions'].append(m)

In [ ]:
for k, v in mentions_dict.items():
    print(len(v))

In [ ]:
pd.DataFrame(mentions_dict)